In [3]:
!pip install pandas
import pandas as pd
print(pd.__version__)  # Должна отобразиться версия, например: 2.1.3

2.0.3


In [1]:
import pandas as pd

# Загрузка данных из CSV
try:
    df = pd.read_csv('/home/max/Drone-Programming/market-vzps.csv')  # Укажите путь к вашему файлу
except FileNotFoundError:
    print("Ошибка: Файл market-vzps.csv не найден!")
    exit()

Ошибка: Файл market-vzps.csv не найден!


In [5]:
print(df.dtypes)

Campaign                object
Campaign №             float64
Cliks                    int64
Spend                  float64
CPC                     object
Conversions phone        int64
Conversions email        int64
Current Conversions      int64
dtype: object


In [7]:
# Удаляем нечисловые символы и конвертируем колонки
cols_to_convert = ['Spend', 'CPC', 'Current Conversions']
for col in cols_to_convert:
    df[col] = pd.to_numeric(
        df[col].astype(str).str.replace('[^\d.]', '', regex=True), 
        errors='coerce'
    )

In [8]:
print(df.dtypes)

Campaign                object
Campaign №             float64
Cliks                    int64
Spend                  float64
CPC                    float64
Conversions phone        int64
Conversions email        int64
Current Conversions      int64
dtype: object


In [10]:
# Заполняем NaN средними значениями
df.fillna({
    'Spend': df['Spend'].mean(),
    'CPC': df['CPC'].mean(),
    'Current Conversions': df['Current Conversions'].median()
}, inplace=True)

In [14]:
# Проверка необходимых колонок
required_columns = ['Campaign', 'Spend', 'CPC', 'Current Conversions']
if not all(col in df.columns for col in required_columns):
    missing = [col for col in required_columns if col not in df.columns]
    print(f"Ошибка: В файле отсутствуют колонки: {missing}")
    exit()

# Удаление строки с итогами (если присутствует)
df = df[df['Campaign'] != 'Всего'].reset_index(drop=True)

# Параметры
total_budget = 990000  # Общий бюджет (можно сделать параметром)

# Равномерное распределение бюджета
n = len(df)
df['Allocated Budget'] = total_budget / n

# Расчет конверсионной ставки (CR)
df['CR'] = (df['Current Conversions'] * df['CPC']) / df['Spend']

# Оптимизация CPC
mean_cr = df['CR'].mean()
min_cpc = 0.3  # Минимальная ставка (можно регулировать)
df['Optimal CPC'] = (df['CPC'] * (mean_cr / df['CR'])).clip(lower=min_cpc)

# Прогноз конверсий
df['Forecast Conversions'] = (df['Allocated Budget'] / df['Optimal CPC']) * df['CR']

# Форматирование результатов
result_df = df[[
    'Campaign',
    'CPC',
    'Optimal CPC',
    'Current Conversions',
    'Forecast Conversions'
]].round(2)

# Вывод результатов
print("Сравнение эффективности кампаний:")
print(result_df.to_string(index=False))
print("\nИтоговая статистика:")
print(f"Общий бюджет: {total_budget}")
print(f"Текущие конверсии: {df['Current Conversions'].sum()}")
print(f"Прогнозные конверсии: {result_df['Forecast Conversions'].sum().round(2)}")
print(f"Ожидаемый рост: {((result_df['Forecast Conversions'].sum() / df['Current Conversions'].sum() - 1) * 100):.2f}%")

Сравнение эффективности кампаний:
                                                         Campaign   CPC  Optimal CPC  Current Conversions  Forecast Conversions
                                            OLD-Ya-ВЗПС-МЭТ-Поиск  1.27         1.76                    4                 24.03
                                         OLD-YA-ВЗПС-Нихром-Поиск 22.87         4.02                   45                 82.96
                                             OLD-ВЗПС-Сетки-Поиск  8.49         3.01                   27                 55.02
                                         OLD-Ya-ВЗПС-Сварка-Поиск 14.65         3.41                   45                 74.16
                                           OLD-YA-ВЗПС-Нихром-РСЯ  4.42         2.35                    8                 47.04
                                            OLD-Ya-ВЗПС-Сетки-РСЯ  5.40         3.08                    8                 33.36
                                           OLD-Ya-ВЗПС-Сварка-РСЯ  3.6

In [22]:
import pandas as pd
import numpy as np

# Конфигурация
MIN_CPC = 0.3
BUDGET_365 = 990000  # Бюджет за 365 дней
BUDGET_180 = 570000  # Бюджет за 180 дней

def load_and_preprocess(file_path: str) -> pd.DataFrame:
    """Загрузка и предобработка данных из CSV файла"""
    df = pd.read_csv(file_path)
    df = df[~df['Campaign'].str.contains('Всего|Total', case=False, na=False)]
    
    numeric_cols = ['Spend', 'CPC', 'Clicks', 'Current Conversions']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace('[^\d.]', '', regex=True),
                errors='coerce'
            ).fillna(0)
    return df

def calculate_optimal_cpc(df: pd.DataFrame, period: str) -> pd.DataFrame:
    """Расчет оптимальных CPC для CR и CPA методов"""
    df = df.copy()
    
    # Расчет CR
    df['CR'] = np.where(
        df['Spend'] > 0,
        (df['Current Conversions'] * df['CPC']) / df['Spend'],
        0
    )
    
    # Расчет CPA
    df['CPA'] = np.where(
        df['Current Conversions'] > 0,
        df['Spend'] / df['Current Conversions'],
        np.inf
    )
    
    # Оптимизация для CR
    valid_cr = df[(df['CR'] > 0) & (df['Current Conversions'] > 0)]
    mean_cr = valid_cr['CR'].mean() if not valid_cr.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CR_{period}'] = np.where(
        df['CR'] > 0,
        (df['CPC'] * (mean_cr / df['CR'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    
    # Оптимизация для CPA
    valid_cpa = df[df['CPA'] < np.inf]
    mean_cpa = valid_cpa['CPA'].mean() if not valid_cpa.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CPA_{period}'] = np.where(
        df['CPA'] < np.inf,
        (df['CPC'] * (mean_cpa / df['CPA'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    
    return df

# Загрузка данных
df_year = calculate_optimal_cpc(
    load_and_preprocess('/home/max/Drone-Programming/market-vzps-365.csv'), 
    '365'
)
df_180 = calculate_optimal_cpc(
    load_and_preprocess('/home/max/Drone-Programming/market-vzps-180.csv'), 
    '180'
)

# Создание отдельных таблиц
cr_table = pd.merge(
    df_year[['Campaign', 'Optimal_CPC_CR_365']],
    df_180[['Campaign', 'Optimal_CPC_CR_180']],
    on='Campaign',
    how='outer'
).fillna(0)

cpa_table = pd.merge(
    df_year[['Campaign', 'Optimal_CPC_CPA_365']],
    df_180[['Campaign', 'Optimal_CPC_CPA_180']],
    on='Campaign',
    how='outer'
).fillna(0)

# Фильтрация кампаний с конверсиями
merged_data = pd.merge(
    df_year[['Campaign', 'Current Conversions']],
    df_180[['Campaign', 'Current Conversions']],
    on='Campaign',
    suffixes=('_365', '_180')
)

conversion_mask = (merged_data['Current Conversions_365'] > 0) | (merged_data['Current Conversions_180'] > 0)
eligible_campaigns = merged_data[conversion_mask]

# Распределение бюджета
n = len(eligible_campaigns)
eligible_campaigns['Allocated_Budget_365'] = BUDGET_365 / n if n > 0 else 0
eligible_campaigns['Allocated_Budget_180'] = BUDGET_180 / n if n > 0 else 0

# Вывод результатов
print("Таблица оптимальных CPC через CR:")
print(cr_table.round(2).to_string(index=False))

print("\nТаблица оптимальных CPC через CPA:")
print(cpa_table.round(2).to_string(index=False))

print("\nРаспределение бюджета:")
print(eligible_campaigns.round(2).to_string(index=False))

Таблица оптимальных CPC через CR:
                                                         Campaign  Optimal_CPC_CR_365  Optimal_CPC_CR_180
                                            OLD-Ya-ВЗПС-МЭТ-Поиск                3.38                0.30
                                         OLD-YA-ВЗПС-Нихром-Поиск                7.73               13.09
                                             OLD-ВЗПС-Сетки-Поиск                5.78                9.60
                                         OLD-Ya-ВЗПС-Сварка-Поиск                6.54               10.88
                                           OLD-YA-ВЗПС-Нихром-РСЯ                4.51                0.30
                                            OLD-Ya-ВЗПС-Сетки-РСЯ                5.92                0.30
                                           OLD-Ya-ВЗПС-Сварка-РСЯ                7.90                0.30
                                              OLD-Ya-ВЗПС-МЭТ-РСЯ               10.83                0.30
            

/tmp/ipykernel_120574/695386060.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_campaigns['Allocated_Budget_365'] = BUDGET_365 / n if n > 0 else 0
/tmp/ipykernel_120574/695386060.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_campaigns['Allocated_Budget_180'] = BUDGET_180 / n if n > 0 else 0


In [23]:
import pandas as pd
import numpy as np

# Конфигурация
MIN_CPC = 0.3
BUDGET_365 = 990000
BUDGET_180 = 570000

def load_and_preprocess(file_path: str) -> pd.DataFrame:
    """Загрузка и предобработка данных"""
    df = pd.read_csv(file_path)
    df = df[~df['Campaign'].str.contains('Всего|Total', case=False, na=False)]
    
    numeric_cols = ['Spend', 'CPC', 'Clicks', 'Current Conversions']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace('[^\d.]', '', regex=True),
                errors='coerce'
            ).fillna(0)
    return df

def calculate_optimal_cpc(df: pd.DataFrame, period: str) -> pd.DataFrame:
    """Расчет оптимальных CPC и конверсий"""
    df = df.copy()
    
    # Расчет CR и CPA
    df['CR'] = np.where(
        df['Spend'] > 0,
        (df['Current Conversions'] * df['CPC']) / df['Spend'],
        0
    )
    df['CPA'] = np.where(
        df['Current Conversions'] > 0,
        df['Spend'] / df['Current Conversions'],
        np.inf
    )
    
    # Оптимизация CPC для CR
    valid_cr = df[(df['CR'] > 0) & (df['Current Conversions'] > 0)]
    mean_cr = valid_cr['CR'].mean() if not valid_cr.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CR_{period}'] = np.where(
        df['CR'] > 0,
        (df['CPC'] * (mean_cr / df['CR'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    
    # Прогноз конверсий для CR
    df[f'Forecast_Conv_CR_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CR_{period}']) * df['CR']
    
    # Оптимизация CPC для CPA
    valid_cpa = df[df['CPA'] < np.inf]
    mean_cpa = valid_cpa['CPA'].mean() if not valid_cpa.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CPA_{period}'] = np.where(
        df['CPA'] < np.inf,
        (df['CPC'] * (mean_cpa / df['CPA'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    
    # Прогноз конверсий для CPA 
    df[f'Forecast_Conv_CPA_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CPA_{period}']) * df['CR']
    
    return df

# Загрузка и расчет данных
df_year = calculate_optimal_cpc(
    load_and_preprocess('/home/max/Drone-Programming/market-vzps-365.csv'), 
    '365'
)
df_180 = calculate_optimal_cpc(
    load_and_preprocess('/home/max/Drone-Programming/market-vzps-180.csv'), 
    '180'
)

# Формирование итоговых таблиц
def create_final_table(method: str) -> pd.DataFrame:
    """Создание таблицы с конверсиями для метода"""
    table = pd.merge(
        df_year[['Campaign', 'Current Conversions', 
                f'Optimal_CPC_{method}_365', f'Forecast_Conv_{method}_365']],
        df_180[['Campaign', 'Current Conversions',
                f'Optimal_CPC_{method}_180', f'Forecast_Conv_{method}_180']],
        on='Campaign',
        suffixes=('_365', '_180')
    )
    
    # Переименование колонок для понятности
    return table.rename(columns={
        'Current Conversions_365': 'Current_Conv_365',
        'Current Conversions_180': 'Current_Conv_180',
        f'Optimal_CPC_{method}_365': 'Optimal_CPC_365',
        f'Optimal_CPC_{method}_180': 'Optimal_CPC_180',
        f'Forecast_Conv_{method}_365': 'Forecast_Conv_365',
        f'Forecast_Conv_{method}_180': 'Forecast_Conv_180'
    })

# Результаты
print("Таблица через CR метод:")
cr_table = create_final_table('CR')
print(cr_table.round(2).to_string(index=False))

print("\nТаблица через CPA метод:")
cpa_table = create_final_table('CPA') 
print(cpa_table.round(2).to_string(index=False))

Таблица через CR метод:
                                                         Campaign  Current_Conv_365  Optimal_CPC_365  Forecast_Conv_365  Current_Conv_180  Optimal_CPC_180  Forecast_Conv_180
                                            OLD-Ya-ВЗПС-МЭТ-Поиск                 4             3.38               1.50                 2             0.30               0.00
                                         OLD-YA-ВЗПС-Нихром-Поиск                45             7.73             133.17                35            13.09              94.92
                                             OLD-ВЗПС-Сетки-Поиск                27             5.78              39.65                20             9.60              24.34
                                         OLD-Ya-ВЗПС-Сварка-Поиск                45             6.54             100.77                29            10.88              49.50
                                           OLD-YA-ВЗПС-Нихром-РСЯ                 8             4.51      

In [1]:
import pandas as pd
import numpy as np

# Конфигурация
MIN_CPC = 0.3
BUDGET_365 = 990000
BUDGET_180 = 570000

def load_and_preprocess(file_path: str) -> pd.DataFrame:
    """Загрузка и предобработка данных"""
    df = pd.read_csv(file_path)
    df = df[~df['Campaign'].str.contains('Всего|Total', case=False, na=False)]
    
    numeric_cols = ['Spend', 'CPC', 'Clicks', 'Current Conversions']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace('[^\d.]', '', regex=True),
                errors='coerce'
            ).fillna(0)
    return df

def calculate_optimal_cpc(df: pd.DataFrame, period: str) -> pd.DataFrame:
    """Расчет оптимальных CPC и конверсий"""
    df = df.copy()
    
    # Расчет метрик
    df['CR'] = np.where(
        df['Spend'] > 0,
        (df['Current Conversions'] * df['CPC']) / df['Spend'],
        0
    )
    df['CPA'] = np.where(
        df['Current Conversions'] > 0,
        df['Spend'] / df['Current Conversions'],
        np.inf
    )
    
    # Оптимизация CPC для CR
    valid_cr = df[(df['CR'] > 0) & (df['Current Conversions'] > 0)]
    mean_cr = valid_cr['CR'].mean() if not valid_cr.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CR_{period}'] = np.where(
        df['CR'] > 0,
        (df['CPC'] * (mean_cr / df['CR'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CR_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CR_{period}']) * df['CR']
    
    # Оптимизация CPC для CPA
    valid_cpa = df[df['CPA'] < np.inf]
    mean_cpa = valid_cpa['CPA'].mean() if not valid_cpa.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CPA_{period}'] = np.where(
        df['CPA'] < np.inf,
        (df['CPC'] * (mean_cpa / df['CPA'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CPA_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CPA_{period}']) * df['CR']
    
    return df

# Загрузка и расчет данных
df_year = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-vzps-365.csv'), 
    '365'
)
df_180 = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-vzps-180.csv'), 
    '180'
)

# Формирование таблицы для CR метода
cr_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_365', 'Forecast_Conv_CR_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_180', 'Forecast_Conv_CR_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Формирование таблицы для CPA метода
cpa_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_365', 'Forecast_Conv_CPA_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_180', 'Forecast_Conv_CPA_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Вывод результатов
print("="*80 + "\nCR Метод:\n" + "="*80)
print(cr_table.round(2).to_string(index=False))

print("\n" + "="*80 + "\nCPA Метод:\n" + "="*80)
print(cpa_table.round(2).to_string(index=False))

CR Метод:
                                                         Campaign  Current_Conv_365  Optimal_CPC_CR_365  Forecast_Conv_CR_365  Current_Conv_180  Optimal_CPC_CR_180  Forecast_Conv_CR_180
                                            OLD-Ya-ВЗПС-МЭТ-Поиск                 4                3.38                  1.50                 2                0.30                  0.00
                                         OLD-YA-ВЗПС-Нихром-Поиск                45                7.73                133.17                35               13.09                 94.92
                                             OLD-ВЗПС-Сетки-Поиск                27                5.78                 39.65                20                9.60                 24.34
                                         OLD-Ya-ВЗПС-Сварка-Поиск                45                6.54                100.77                29               10.88                 49.50
                                           OLD-YA-ВЗПС-Нихро

In [2]:
from IPython.display import HTML

def print_table(df, title):
    """Вывод таблицы с уменьшенным шрифтом"""
    html = f"<h3>{title}</h3>"
    html += df.round(2).style\
              .set_table_styles([{
                  'selector': 'td, th',
                  'props': [('font-size', '85%'), 
                          ('padding', '4px 8px')]
              }])\
              .set_properties(**{'border': '1px solid #ddd'})\
              .hide(axis='index')\
              .to_html()
    display(HTML(html))

# Вывод таблиц с новым стилем
print_table(cr_table, "CR Метод")
print_table(cpa_table, "CPA Метод")

Campaign,Current_Conv_365,Optimal_CPC_CR_365,Forecast_Conv_CR_365,Current_Conv_180,Optimal_CPC_CR_180,Forecast_Conv_CR_180
OLD-Ya-ВЗПС-МЭТ-Поиск,4,3.380000,1.500000,2,0.300000,0.000000
OLD-YA-ВЗПС-Нихром-Поиск,45,7.730000,133.170000,35,13.090000,94.920000
OLD-ВЗПС-Сетки-Поиск,27,5.780000,39.650000,20,9.600000,24.340000
OLD-Ya-ВЗПС-Сварка-Поиск,45,6.540000,100.770000,29,10.880000,49.500000
OLD-YA-ВЗПС-Нихром-РСЯ,8,4.510000,7.840000,1,0.300000,0.000000
OLD-Ya-ВЗПС-Сетки-РСЯ,8,5.920000,7.300000,0,0.300000,0.000000
OLD-Ya-ВЗПС-Сварка-РСЯ,4,7.900000,1.840000,0,0.300000,0.000000
OLD-Ya-ВЗПС-МЭТ-РСЯ,12,10.830000,12.320000,0,0.300000,0.000000
LMA-YA-ВЗПС-Проволока-Поиск,82,18.530000,281.180000,48,32.600000,113.310000
LMA-YA-ВЗПС-Пруток нержавеющий-Поиск,7,109.260000,4.360000,7,97.260000,5.510000


Campaign,Current_Conv_365,Optimal_CPC_CPA_365,Forecast_Conv_CPA_365,Current_Conv_180,Optimal_CPC_CPA_180,Forecast_Conv_CPA_180
OLD-Ya-ВЗПС-МЭТ-Поиск,4,7.930000,0.640000,2,nan,nan
OLD-YA-ВЗПС-Нихром-Поиск,45,62.530000,16.460000,35,82.090000,15.130000
OLD-ВЗПС-Сетки-Поиск,27,31.030000,7.390000,20,36.840000,6.340000
OLD-Ya-ВЗПС-Сварка-Поиск,45,47.320000,13.930000,29,51.670000,10.420000
OLD-YA-ВЗПС-Нихром-РСЯ,8,20.700000,1.710000,1,nan,nan
OLD-Ya-ВЗПС-Сетки-РСЯ,8,19.270000,2.240000,0,0.300000,0.000000
OLD-Ya-ВЗПС-Сварка-РСЯ,4,9.720000,1.490000,0,0.300000,0.000000
OLD-Ya-ВЗПС-МЭТ-РСЯ,12,21.700000,6.150000,0,0.300000,0.000000
LMA-YA-ВЗПС-Проволока-Поиск,82,72.460000,71.900000,48,71.450000,51.690000
LMA-YA-ВЗПС-Пруток нержавеющий-Поиск,7,13.170000,36.190000,7,23.810000,22.490000


In [20]:
import pandas as pd
import numpy as np

# Конфигурация
MIN_CPC = 0.3
BUDGET_365 = 357000
BUDGET_180 = 215000

def load_and_preprocess(file_path: str) -> pd.DataFrame:
    """Загрузка и предобработка данных"""
    df = pd.read_csv(file_path, sep='\t')
    df = df[~df['Campaign'].str.contains('Всего|Total', case=False, na=False)]
    
    numeric_cols = ['Spend', 'CPC', 'Clicks', 'Current Conversions']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace('[^\d.]', '', regex=True),
                errors='coerce'
            ).fillna(0)
    return df

def calculate_optimal_cpc(df: pd.DataFrame, period: str) -> pd.DataFrame:
    """Расчет оптимальных CPC и конверсий"""
    df = df.copy()
    
    # Расчет метрик
    df['CR'] = np.where(
        df['Spend'] > 0,
        (df['Current Conversions'] * df['CPC']) / df['Spend'],
        0
    )
    df['CPA'] = np.where(
        df['Current Conversions'] > 0,
        df['Spend'] / df['Current Conversions'],
        np.inf
    )
    
    # Оптимизация CPC для CR
    valid_cr = df[(df['CR'] > 0) & (df['Current Conversions'] > 0)]
    mean_cr = valid_cr['CR'].mean() if not valid_cr.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CR_{period}'] = np.where(
        df['CR'] > 0,
        (df['CPC'] * (mean_cr / df['CR'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CR_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CR_{period}']) * df['CR']
    
    # Оптимизация CPC для CPA
    valid_cpa = df[df['CPA'] < np.inf]
    mean_cpa = valid_cpa['CPA'].mean() if not valid_cpa.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CPA_{period}'] = np.where(
        df['CPA'] < np.inf,
        (df['CPC'] * (mean_cpa / df['CPA'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CPA_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CPA_{period}']) * df['CR']
    
    return df

# Загрузка и расчет данных
df_year = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-vzm-365.csv'), 
    '365'
)
df_180 = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-vzm-180.csv'), 
    '180'
)

# Формирование таблицы для CR метода
cr_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_365', 'Forecast_Conv_CR_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_180', 'Forecast_Conv_CR_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Формирование таблицы для CPA метода
cpa_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_365', 'Forecast_Conv_CPA_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_180', 'Forecast_Conv_CPA_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Вывод результатов
print("="*80 + "\nCR Метод:\n" + "="*80)
print(cr_table.round(2).to_string(index=False))

print("\n" + "="*80 + "\nCPA Метод:\n" + "="*80)
print(cpa_table.round(2).to_string(index=False))

CR Метод:
                                           Campaign  Current_Conv_365  Optimal_CPC_CR_365  Forecast_Conv_CR_365  Current_Conv_180  Optimal_CPC_CR_180  Forecast_Conv_CR_180
                       OLD-Ya-ВЗМ-Металлорукава-РСЯ                 5               12.14                  2.39                 0                0.30                  0.00
                        OLD-Ya-ВЗМ-Компенсаторы-РСЯ                 2               13.00                  0.26                 0                0.30                  0.00
                     OLD-Ya-ВЗМ-Металлорукава-Поиск                52               10.15                 91.07                 4               15.75                  4.21
                      OLD-Ya-ВЗМ-Компенсаторы-Поиск                19               16.65                 29.68                 3               22.97                  4.53
                  OLD-Ya-ВЗМ-Конкуренты-Поиск и РСЯ                 4               16.13                  0.69                 1 

In [21]:
from IPython.display import HTML

def print_table(df, title):
    """Вывод таблицы с уменьшенным шрифтом"""
    html = f"<h3>{title}</h3>"
    html += df.round(2).style\
              .set_table_styles([{
                  'selector': 'td, th',
                  'props': [('font-size', '85%'), 
                          ('padding', '4px 8px')]
              }])\
              .set_properties(**{'border': '1px solid #ddd'})\
              .hide(axis='index')\
              .to_html()
    display(HTML(html))

# Вывод таблиц с новым стилем
print_table(cr_table, "CR Метод")
print_table(cpa_table, "CPA Метод")

Campaign,Current_Conv_365,Optimal_CPC_CR_365,Forecast_Conv_CR_365,Current_Conv_180,Optimal_CPC_CR_180,Forecast_Conv_CR_180
OLD-Ya-ВЗМ-Металлорукава-РСЯ,5,12.140000,2.390000,0,0.300000,0.000000
OLD-Ya-ВЗМ-Компенсаторы-РСЯ,2,13.000000,0.260000,0,0.300000,0.000000
OLD-Ya-ВЗМ-Металлорукава-Поиск,52,10.150000,91.070000,4,15.750000,4.210000
OLD-Ya-ВЗМ-Компенсаторы-Поиск,19,16.650000,29.680000,3,22.970000,4.530000
OLD-Ya-ВЗМ-Конкуренты-Поиск и РСЯ,4,16.130000,0.690000,1,10.610000,0.450000
LMA-YA-ВЗМ-Гибкие трубопроводы - Поиск,6,13.080000,5.720000,4,3.530000,9.990000
LMA-YA-ВЗМ-Металлогофр - Поиск,6,22.350000,3.450000,5,22.520000,3.270000
LMA-YA-ВЗМ-Шланги нержавеющие-Поиск,15,26.760000,14.470000,9,26.540000,8.690000
LMA-YA-ВЗМ-Гибкие трубки и трубы-Поиск,4,58.240000,1.920000,2,102.190000,0.600000
LMA-YA-ВЗМ-Рукава нержавеющие-Поиск,20,19.690000,20.960000,5,35.750000,2.750000


Campaign,Current_Conv_365,Optimal_CPC_CPA_365,Forecast_Conv_CPA_365,Current_Conv_180,Optimal_CPC_CPA_180,Forecast_Conv_CPA_180
OLD-Ya-ВЗМ-Металлорукава-РСЯ,5,7.130000,4.070000,0,0.300000,0.000000
OLD-Ya-ВЗМ-Компенсаторы-РСЯ,2,1.920000,1.740000,0,0.300000,0.000000
OLD-Ya-ВЗМ-Металлорукава-Поиск,52,26.130000,35.360000,4,20.850000,3.180000
OLD-Ya-ВЗМ-Компенсаторы-Поиск,19,23.310000,21.200000,3,29.910000,3.480000
OLD-Ya-ВЗМ-Конкуренты-Поиск и РСЯ,4,2.570000,4.320000,1,8.850000,0.540000
LMA-YA-ВЗМ-Гибкие трубопроводы - Поиск,6,14.230000,5.260000,4,49.470000,0.710000
LMA-YA-ВЗМ-Металлогофр - Поиск,6,8.590000,8.990000,5,12.960000,5.690000
LMA-YA-ВЗМ-Шланги нержавеющие-Поиск,15,14.390000,26.900000,9,19.130000,12.060000
LMA-YA-ВЗМ-Гибкие трубки и трубы-Поиск,4,7.160000,15.610000,2,5.990000,10.320000
LMA-YA-ВЗМ-Рукава нержавеющие-Поиск,20,15.640000,26.390000,5,10.900000,9.030000


In [34]:
import pandas as pd
import numpy as np

# Конфигурация
MIN_CPC = 0.3
BUDGET_365 = 245000
BUDGET_180 = 180000

def load_and_preprocess(file_path: str) -> pd.DataFrame:
    """Загрузка и предобработка данных"""
    df = pd.read_csv(file_path, sep='\t')
    df = df[~df['Campaign'].str.contains('Всего|Total', case=False, na=False)]
    
    numeric_cols = ['Spend', 'CPC', 'Clicks', 'Current Conversions']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace('[^\d.]', '', regex=True),
                errors='coerce'
            ).fillna(0)
    return df

def calculate_optimal_cpc(df: pd.DataFrame, period: str) -> pd.DataFrame:
    """Расчет оптимальных CPC и конверсий"""
    df = df.copy()
    
    # Расчет метрик
    df['CR'] = np.where(
        df['Spend'] > 0,
        (df['Current Conversions'] * df['CPC']) / df['Spend'],
        0
    )
    df['CPA'] = np.where(
        df['Current Conversions'] > 0,
        df['Spend'] / df['Current Conversions'],
        np.inf
    )
    
    # Оптимизация CPC для CR
    valid_cr = df[(df['CR'] > 0) & (df['Current Conversions'] > 0)]
    mean_cr = valid_cr['CR'].mean() if not valid_cr.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CR_{period}'] = np.where(
        df['CR'] > 0,
        (df['CPC'] * (mean_cr / df['CR'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CR_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CR_{period}']) * df['CR']
    
    # Оптимизация CPC для CPA
    valid_cpa = df[df['CPA'] < np.inf]
    mean_cpa = valid_cpa['CPA'].mean() if not valid_cpa.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CPA_{period}'] = np.where(
        df['CPA'] < np.inf,
        (df['CPC'] * (mean_cpa / df['CPA'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CPA_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CPA_{period}']) * df['CR']
    
    return df

# Загрузка и расчет данных
df_year = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-magn-365.csv'), 
    '365'
)
df_180 = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-magn-180.csv'), 
    '180'
)

# Формирование таблицы для CR метода
cr_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_365', 'Forecast_Conv_CR_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_180', 'Forecast_Conv_CR_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Формирование таблицы для CPA метода
cpa_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_365', 'Forecast_Conv_CPA_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_180', 'Forecast_Conv_CPA_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Вывод результатов
print("="*80 + "\nCR Метод:\n" + "="*80)
print(cr_table.round(2).to_string(index=False))

print("\n" + "="*80 + "\nCPA Метод:\n" + "="*80)
print(cpa_table.round(2).to_string(index=False))

CR Метод:
                                               Campaign  Current_Conv_365  Optimal_CPC_CR_365  Forecast_Conv_CR_365  Current_Conv_180  Optimal_CPC_CR_180  Forecast_Conv_CR_180
                             OLD-Ya-Магнетон П-ПУ-Поиск                40                6.65                 78.97                32               31.15                 16.02
       LMA-YA-Магнетон П-Конкуренты производители-Поиск                33               14.74                 62.26                23               70.46                 10.01
                          LMA-YA-Магнетон П-Общая-Поиск                38               23.64                 50.83                29               93.90                 11.29
                           LMA-YA-Магнетон П-Сита-Поиск                18               22.88                 27.90                15               86.17                  7.41
              LMA-YA-Магнетон П-Ролики и барабаны-Поиск                16               23.86                 

In [35]:
from IPython.display import HTML

def print_table(df, title):
    """Вывод таблицы с уменьшенным шрифтом"""
    html = f"<h3>{title}</h3>"
    html += df.round(2).style\
              .set_table_styles([{
                  'selector': 'td, th',
                  'props': [('font-size', '85%'), 
                          ('padding', '4px 8px')]
              }])\
              .set_properties(**{'border': '1px solid #ddd'})\
              .hide(axis='index')\
              .to_html()
    display(HTML(html))

# Вывод таблиц с новым стилем
print_table(cr_table, "CR Метод")
print_table(cpa_table, "CPA Метод")

Campaign,Current_Conv_365,Optimal_CPC_CR_365,Forecast_Conv_CR_365,Current_Conv_180,Optimal_CPC_CR_180,Forecast_Conv_CR_180
OLD-Ya-Магнетон П-ПУ-Поиск,40,6.650000,78.970000,32,31.150000,16.020000
LMA-YA-Магнетон П-Конкуренты производители-Поиск,33,14.740000,62.260000,23,70.460000,10.010000
LMA-YA-Магнетон П-Общая-Поиск,38,23.640000,50.830000,29,93.900000,11.290000
LMA-YA-Магнетон П-Сита-Поиск,18,22.880000,27.900000,15,86.170000,7.410000
LMA-YA-Магнетон П-Ролики и барабаны-Поиск,16,23.860000,18.280000,14,90.850000,5.080000
LMA-YA-Магнетон П-Футерованные изделия-Поиск,7,20.780000,8.420000,5,73.700000,1.800000
LMA-YA-Магнетон П-Листы полиуретановые-Поиск,7,20.560000,6.710000,5,66.950000,1.550000
LMA-YA-Магнетон П-Услуги по футеровке-Поиск,6,24.800000,5.870000,3,147.740000,0.540000
LMA-YA-Магнетон П-Изделия на заказ-Поиск,48,15.140000,98.370000,34,61.700000,19.560000
LMA-YA-Магнетон ЛЦ-Отливки и слитки-Поиск,7,10.920000,9.040000,6,13.060000,3.840000


Campaign,Current_Conv_365,Optimal_CPC_CPA_365,Forecast_Conv_CPA_365,Current_Conv_180,Optimal_CPC_CPA_180,Forecast_Conv_CPA_180
OLD-Ya-Магнетон П-ПУ-Поиск,40,24.160000,21.720000,32,30.200000,16.530000
LMA-YA-Магнетон П-Конкуренты производители-Поиск,33,23.090000,39.760000,23,26.260000,26.870000
LMA-YA-Магнетон П-Общая-Поиск,38,16.370000,73.400000,29,23.470000,45.160000
LMA-YA-Магнетон П-Сита-Поиск,18,18.970000,33.650000,15,29.790000,21.430000
LMA-YA-Магнетон П-Ролики и барабаны-Поиск,16,13.980000,31.190000,14,21.890000,21.090000
LMA-YA-Магнетон П-Футерованные изделия-Поиск,7,14.730000,11.880000,5,21.750000,6.110000
LMA-YA-Магнетон П-Листы полиуретановые-Поиск,7,11.730000,11.760000,5,18.720000,5.550000
LMA-YA-Магнетон П-Услуги по футеровке-Поиск,6,11.970000,12.160000,3,10.810000,7.350000
LMA-YA-Магнетон П-Изделия на заказ-Поиск,48,25.080000,59.370000,34,34.700000,34.790000
LMA-YA-Магнетон ЛЦ-Отливки и слитки-Поиск,7,15.800000,6.250000,6,38.600000,1.300000


In [30]:
import pandas as pd
import numpy as np

# Конфигурация
MIN_CPC = 0.3
BUDGET_365 = 86000
BUDGET_180 = 44000

def load_and_preprocess(file_path: str) -> pd.DataFrame:
    """Загрузка и предобработка данных"""
    df = pd.read_csv(file_path, sep='\t')
    df = df[~df['Campaign'].str.contains('Всего|Total', case=False, na=False)]
    
    numeric_cols = ['Spend', 'CPC', 'Clicks', 'Current Conversions']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace('[^\d.]', '', regex=True),
                errors='coerce'
            ).fillna(0)
    return df

def calculate_optimal_cpc(df: pd.DataFrame, period: str) -> pd.DataFrame:
    """Расчет оптимальных CPC и конверсий"""
    df = df.copy()
    
    # Расчет метрик
    df['CR'] = np.where(
        df['Spend'] > 0,
        (df['Current Conversions'] * df['CPC']) / df['Spend'],
        0
    )
    df['CPA'] = np.where(
        df['Current Conversions'] > 0,
        df['Spend'] / df['Current Conversions'],
        np.inf
    )
    
    # Оптимизация CPC для CR
    valid_cr = df[(df['CR'] > 0) & (df['Current Conversions'] > 0)]
    mean_cr = valid_cr['CR'].mean() if not valid_cr.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CR_{period}'] = np.where(
        df['CR'] > 0,
        (df['CPC'] * (mean_cr / df['CR'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CR_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CR_{period}']) * df['CR']
    
    # Оптимизация CPC для CPA
    valid_cpa = df[df['CPA'] < np.inf]
    mean_cpa = valid_cpa['CPA'].mean() if not valid_cpa.empty else df['CPC'].mean()
    df[f'Optimal_CPC_CPA_{period}'] = np.where(
        df['CPA'] < np.inf,
        (df['CPC'] * (mean_cpa / df['CPA'])).clip(lower=MIN_CPC),
        MIN_CPC
    )
    df[f'Forecast_Conv_CPA_{period}'] = (df['Spend'] / df[f'Optimal_CPC_CPA_{period}']) * df['CR']
    
    return df

# Загрузка и расчет данных
df_year = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-optimus-365.csv'), 
    '365'
)
df_180 = calculate_optimal_cpc(
    load_and_preprocess('/home/max/umcom/promotion/market-optimus-180.csv'), 
    '180'
)

# Формирование таблицы для CR метода
cr_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_365', 'Forecast_Conv_CR_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CR_180', 'Forecast_Conv_CR_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Формирование таблицы для CPA метода
cpa_table = pd.merge(
    df_year[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_365', 'Forecast_Conv_CPA_365']],
    df_180[['Campaign', 'Current Conversions', 'Optimal_CPC_CPA_180', 'Forecast_Conv_CPA_180']],
    on='Campaign',
    suffixes=('_365', '_180')
).rename(columns={
    'Current Conversions_365': 'Current_Conv_365',
    'Current Conversions_180': 'Current_Conv_180'
})

# Вывод результатов
print("="*80 + "\nCR Метод:\n" + "="*80)
print(cr_table.round(2).to_string(index=False))

print("\n" + "="*80 + "\nCPA Метод:\n" + "="*80)
print(cpa_table.round(2).to_string(index=False))

CR Метод:
                                   Campaign  Current_Conv_365  Optimal_CPC_CR_365  Forecast_Conv_CR_365  Current_Conv_180  Optimal_CPC_CR_180  Forecast_Conv_CR_180
            LMA-YA-Оптимус-Нержавеющий лист                 2               32.51                  2.65                 0                0.30                  0.00
      LMA-YA-Оптимус-Нержавеющий лист (+ЯК)                 2               23.67                  1.45                 2             1336.06                  2.64
     LMA-YA-Оптимус-Нержавеющая лента (+ЯК)                 3                7.89                  5.63                 3              352.25                 12.49
      LMA-YA-Оптимус-Нержавеющий лист - РСЯ                 4                8.09                  0.21                 1             1345.54                  0.02
     LMA-YA-Оптимус-Нержавеющая лента - РСЯ                 9                2.43                  2.45                 5               65.64                  2.06
      

In [31]:
from IPython.display import HTML

def print_table(df, title):
    """Вывод таблицы с уменьшенным шрифтом"""
    html = f"<h3>{title}</h3>"
    html += df.round(2).style\
              .set_table_styles([{
                  'selector': 'td, th',
                  'props': [('font-size', '85%'), 
                          ('padding', '4px 8px')]
              }])\
              .set_properties(**{'border': '1px solid #ddd'})\
              .hide(axis='index')\
              .to_html()
    display(HTML(html))

# Вывод таблиц с новым стилем
print_table(cr_table, "CR Метод")
print_table(cpa_table, "CPA Метод")

Campaign,Current_Conv_365,Optimal_CPC_CR_365,Forecast_Conv_CR_365,Current_Conv_180,Optimal_CPC_CR_180,Forecast_Conv_CR_180
LMA-YA-Оптимус-Нержавеющий лист,2,32.510000,2.650000,0,0.300000,0.000000
LMA-YA-Оптимус-Нержавеющий лист (+ЯК),2,23.670000,1.450000,2,1336.060000,2.640000
LMA-YA-Оптимус-Нержавеющая лента (+ЯК),3,7.890000,5.630000,3,352.250000,12.490000
LMA-YA-Оптимус-Нержавеющий лист - РСЯ,4,8.090000,0.210000,1,1345.540000,0.020000
LMA-YA-Оптимус-Нержавеющая лента - РСЯ,9,2.430000,2.450000,5,65.640000,2.060000
LMA-YA-Оптимус-Нержавеющий лист - LAL,1,14.630000,0.060000,0,0.300000,0.000000
LMA-YA-Оптимус-Нержавеющая лента - LAL,19,7.300000,2.730000,18,469.010000,4.030000
LMA-YA-Оптимус-Нержавеющий лист - РПиП РСЯ,1,3.920000,0.060000,0,0.300000,0.000000
LMA-YA-Оптимус-Нержавеющая лента - РПиП РСЯ,6,3.410000,1.600000,6,216.630000,3.460000
LMA-YA-Оптимус-Нержавеющий лист - ВП РСЯ,4,0.980000,0.820000,1,0.300000,0.000000


Campaign,Current_Conv_365,Optimal_CPC_CPA_365,Forecast_Conv_CPA_365,Current_Conv_180,Optimal_CPC_CPA_180,Forecast_Conv_CPA_180
LMA-YA-Оптимус-Нержавеющий лист,2,12.600000,6.840000,0,0.300000,0.000000
LMA-YA-Оптимус-Нержавеющий лист (+ЯК),2,6.890000,4.980000,2,652.510000,5.410000
LMA-YA-Оптимус-Нержавеющая лента (+ЯК),3,17.840000,2.490000,3,2054.500000,2.140000
LMA-YA-Оптимус-Нержавеющий лист - РСЯ,4,0.490000,3.400000,1,11.740000,2.730000
LMA-YA-Оптимус-Нержавеющая лента - РСЯ,9,2.590000,2.300000,5,203.070000,0.660000
LMA-YA-Оптимус-Нержавеющий лист - LAL,1,0.620000,1.540000,0,0.300000,0.000000
LMA-YA-Оптимус-Нержавеющая лента - LAL,19,1.370000,14.590000,18,110.520000,17.100000
LMA-YA-Оптимус-Нержавеющий лист - РПиП РСЯ,1,0.530000,0.410000,0,0.300000,0.000000
LMA-YA-Оптимус-Нержавеющая лента - РПиП РСЯ,6,2.530000,2.150000,6,284.850000,2.630000
LMA-YA-Оптимус-Нержавеющий лист - ВП РСЯ,4,1.940000,0.410000,1,nan,nan
